# Objective: Estimating the Target Recovery with less Significant Features

## Part 1: Dealing with numeric variables

### 1A: Cleaning the numeric variables

In [1]:
#Data Manipulation packages
import numpy as np
import pandas as pd

#For correlation
import seaborn as sns

#Data visualization package
import plotly as ply
import matplotlib.pyplot as plt

#Dara Preprocessing package
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

#Model development package
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#Model Evaluation package
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

In [2]:
# Reading the final csv after SQL work
data = pd.read_csv('/Users/varunbhavnani/Documents/Crescent Bank DS Competition/New_Final.csv')
print(data.shape)
data.info()

(13718, 94)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Data columns (total 94 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   masked_acct                                          13718 non-null  object 
 1   key_1                                                13718 non-null  object 
 2   Vintage                                              13718 non-null  int64  
 3   ChargeOffMonthKey                                    13718 non-null  int64  
 4   ChargeOffMOB                                         13718 non-null  int64  
 5   Loss_Date_150                                        13718 non-null  object 
 6   BalanceAtDefault                                     13718 non-null  float64
 7   JointIndicator                                       13718 non-null  int64  
 8   LCPIndicator                                         1

In [3]:
# Keeping the categorical variables & ids aside
cat_var = data[['StateApplicant', 'EmploymentJobTypeApplicant', 'BackendType', 'VehicleModelNADA', 'VehicleMakeNADA', 'Loss_Date_150']]

ids = data[['masked_acct', 'key_1','key_1:1', 'key_2','key_2:1','masked_acct:1']]

df = data.drop(['masked_acct', 'key_1','key_1:1', 'key_2','key_2:1','masked_acct:1','StateApplicant', 'EmploymentJobTypeApplicant', 'BackendType', 'VehicleModelNADA', 'VehicleMakeNADA', 'Loss_Date_150'], axis = 1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Data columns (total 82 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Vintage                                              13718 non-null  int64  
 1   ChargeOffMonthKey                                    13718 non-null  int64  
 2   ChargeOffMOB                                         13718 non-null  int64  
 3   BalanceAtDefault                                     13718 non-null  float64
 4   JointIndicator                                       13718 non-null  int64  
 5   LCPIndicator                                         13718 non-null  int64  
 6   FICOScore                                            9504 non-null   float64
 7   FICOScorePctAvgFICOLast30DaysBookedLoans             9504 non-null   float64
 8   ApplicationWeekday                                   13718 non-nul

In [4]:
# Dropping variables with too many NA values

df = df.drop(['CreditBureauRevolvingCreditLinePctUsed','TradeInPctLine5','TradeInPctLine4','TradeInPctLine3','TradeInAmt', 'TradeInPctBBVehicleValue'], axis = 1)

In [5]:
df['FICOScore'] = df['FICOScore'].fillna(df['FICOScore'].mean())
df['FICOScorePctAvgFICOLast30DaysBookedLoans'] = df['FICOScorePctAvgFICOLast30DaysBookedLoans'].fillna(df['FICOScorePctAvgFICOLast30DaysBookedLoans'].mean())
df['NumExceptionPctAvgNumExceptionLast30DaysBookedLoans'] = df['NumExceptionPctAvgNumExceptionLast30DaysBookedLoans'].fillna(df['NumExceptionPctAvgNumExceptionLast30DaysBookedLoans'].mean())
df['CreditBureauPctTradesDelq'] = df['CreditBureauPctTradesDelq'].fillna(df['CreditBureauPctTradesDelq'].mean())
df['TotalDownPmtPctBBVehicleValue'] = df['TotalDownPmtPctBBVehicleValue'].fillna(df['TotalDownPmtPctBBVehicleValue'].mean())

df['BackendTotalPctBBVehicleValue'] = df['BackendTotalPctBBVehicleValue'].fillna(df['BackendTotalPctBBVehicleValue'].mean())
df['CashDownPmtPctBBVehicleValue'] = df['CashDownPmtPctBBVehicleValue'].fillna(df['CashDownPmtPctBBVehicleValue'].mean())
df['LTVLine3'] = df['LTVLine3'].fillna(df['LTVLine3'].mean())
df['LTVLine4'] = df['LTVLine4'].fillna(df['LTVLine4'].mean())
df['LTVLine5'] = df['LTVLine5'].fillna(df['LTVLine5'].mean())
df['LTVLine3ExclRebate'] = df['LTVLine3ExclRebate'].fillna(df['LTVLine3ExclRebate'].mean())
df['LTVLine4ExclRebate'] = df['LTVLine4ExclRebate'].fillna(df['LTVLine4ExclRebate'].mean())
df['LTVLine5ExclRebate'] = df['LTVLine5ExclRebate'].fillna(df['LTVLine5ExclRebate'].mean())
df['MoodysUnemploymentRate'] = df['MoodysUnemploymentRate'].fillna(df['MoodysUnemploymentRate'].mean())
df['MoodysNumLaborForce'] = df['MoodysNumLaborForce'].fillna(df['MoodysNumLaborForce'].mean())
df['MoodysGDPReal'] = df['MoodysGDPReal'].fillna(df['MoodysGDPReal'].mean())
df['MoodysNumEmployed'] = df['MoodysNumEmployed'].fillna(df['MoodysNumEmployed'].mean())
df['MoodysNumUnemployed'] = df['MoodysNumUnemployed'].fillna(df['MoodysNumUnemployed'].mean())
df['VehicleValueBlackBook'] = df['VehicleValueBlackBook'].fillna(df['VehicleValueBlackBook'].mean())

In [6]:
# Complete variables with NAs
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Data columns (total 76 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Vintage                                              13718 non-null  int64  
 1   ChargeOffMonthKey                                    13718 non-null  int64  
 2   ChargeOffMOB                                         13718 non-null  int64  
 3   BalanceAtDefault                                     13718 non-null  float64
 4   JointIndicator                                       13718 non-null  int64  
 5   LCPIndicator                                         13718 non-null  int64  
 6   FICOScore                                            13718 non-null  float64
 7   FICOScorePctAvgFICOLast30DaysBookedLoans             13718 non-null  float64
 8   ApplicationWeekday                                   13718 non-nul

In [7]:
# Scaling the data
from scipy import stats

df_scaled = stats.zscore(df)

df_scaled.head()

,Vintage,ChargeOffMonthKey,ChargeOffMOB,BalanceAtDefault,JointIndicator,LCPIndicator,FICOScore,FICOScorePctAvgFICOLast30DaysBookedLoans,ApplicationWeekday,NewUsedIndicator,...,NumExceptionDTI,NumExceptionPctAvgNumExceptionLast30DaysBookedLoans,NumPriorCrescentApplications,NumPriorCrescentApplicationsLast30Days,NumPriorCrescentApplicationsLast60Days,NumPriorCrescentApplicationsLast90Days,NumPriorCrescentApplicationsLast180Days,JDPUVIDiffCOFromOrig,MoodysUVIDiffCOFromOrig,True_Recovery
0,1.698842,1.067574,-0.299149,-1.298721,-0.498678,2.09376,0.348685,0.486393,-0.312812,-0.689011,...,-0.105499,0.082546,-0.554403,-0.434474,-0.475715,-0.486963,-0.497917,2.181875,2.043525,-1.110074
1,2.934303,1.067574,-1.023056,1.439201,-0.498678,-0.47761,3.340728,3.631088,-1.442510,-0.689011,...,-0.105499,-0.781028,-0.554403,-0.434474,-0.475715,-0.486963,-0.497917,2.020707,1.798437,0.437607
2,-0.993162,-0.158740,0.941835,-1.617047,-0.498678,-0.47761,0.323112,-0.060089,0.816886,-0.689011,...,-0.105499,-0.781028,-0.554403,-0.434474,-0.475715,-0.486963,-0.497917,-1.766859,-0.764104,-1.911105
3,-0.941142,-1.224601,0.011097,-0.061589,-0.498678,-0.47761,-1.390280,-1.829585,1.381735,1.451355,...,-0.105499,0.631031,0.352177,-0.434474,1.033155,0.957395,0.821207,-1.024394,-0.725979,-0.363424
4,1.737857,1.021731,-1.023056,-0.187354,-0.498678,-0.47761,-0.674235,-0.576135,-0.312812,1.451355,...,-0.105499,-0.781028,0.352177,-0.434474,-0.475715,-0.486963,0.821207,0.945700,1.545178,0.630527


### 1B: Feature Engineering

In [8]:
#Dependent Variable
Y = df_scaled[['True_Recovery']]

# Independent Variables
X = df_scaled.iloc[:,:-1]

In [9]:
#Check the correlation between explanatory variables

x_corr = X.corr()

x_corr

,Vintage,ChargeOffMonthKey,ChargeOffMOB,BalanceAtDefault,JointIndicator,LCPIndicator,FICOScore,FICOScorePctAvgFICOLast30DaysBookedLoans,ApplicationWeekday,NewUsedIndicator,...,NumExceptionPTI,NumExceptionDTI,NumExceptionPctAvgNumExceptionLast30DaysBookedLoans,NumPriorCrescentApplications,NumPriorCrescentApplicationsLast30Days,NumPriorCrescentApplicationsLast60Days,NumPriorCrescentApplicationsLast90Days,NumPriorCrescentApplicationsLast180Days,JDPUVIDiffCOFromOrig,MoodysUVIDiffCOFromOrig
Vintage,1.000000,0.482234,-0.373906,0.069838,-0.040601,0.237970,-0.120693,0.014981,0.001470,-0.101044,...,0.041371,0.037320,-0.013582,0.045665,0.008555,0.006590,0.006672,0.005734,0.740714,0.555446
ChargeOffMonthKey,0.482234,1.000000,0.538444,-0.139319,0.003360,0.147552,-0.035757,0.037495,0.007375,-0.061702,...,0.019751,0.010430,-0.014751,0.023520,0.005596,0.001856,0.001745,-0.000764,0.277897,-0.141056
ChargeOffMOB,-0.373906,0.538444,1.000000,-0.259929,0.040625,-0.094357,0.076359,0.029767,0.011792,-0.004924,...,-0.019129,-0.028742,-0.003395,-0.018356,-0.006236,-0.007643,-0.006737,-0.009381,-0.310546,-0.652525
BalanceAtDefault,0.069838,-0.139319,-0.259929,1.000000,0.128268,-0.098761,0.023492,0.025445,0.013129,0.448641,...,0.007248,0.014456,-0.079138,0.067409,0.001621,0.001986,0.005165,0.008644,0.063600,0.162834
JointIndicator,-0.040601,0.003360,0.040625,0.128268,1.000000,-0.012412,-0.012523,0.025499,0.019283,0.094755,...,-0.050187,0.034862,0.028317,0.149974,0.106520,0.107869,0.107015,0.109262,-0.038264,-0.045206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NumPriorCrescentApplicationsLast60Days,0.006590,0.001856,-0.007643,0.001986,0.107869,-0.013104,-0.030652,-0.026260,-0.036046,-0.003596,...,0.001422,0.007794,0.006239,0.675691,0.900362,1.000000,0.969385,0.903420,0.004553,0.007698
NumPriorCrescentApplicationsLast90Days,0.006672,0.001745,-0.006737,0.005165,0.107015,-0.014364,-0.029628,-0.025695,-0.036713,-0.000191,...,-0.000472,0.006146,0.000803,0.702121,0.866195,0.969385,1.000000,0.938372,0.004168,0.008219
NumPriorCrescentApplicationsLast180Days,0.005734,-0.000764,-0.009381,0.008644,0.109262,-0.018488,-0.026865,-0.022844,-0.035662,0.008466,...,-0.000983,0.005533,-0.002716,0.762752,0.801614,0.903420,0.938372,1.000000,0.001080,0.010909
JDPUVIDiffCOFromOrig,0.740714,0.277897,-0.310546,0.063600,-0.038264,0.168333,-0.072959,0.016405,0.008700,-0.077443,...,0.019472,0.023887,-0.001358,0.033397,0.005332,0.004553,0.004168,0.001080,1.000000,0.735234


In [10]:
upper_tri = x_corr.where(np.triu(np.ones(x_corr.shape),k=1).astype(np.bool))
print(upper_tri)

                                         Vintage  ChargeOffMonthKey  \
Vintage                                      NaN           0.482234   
ChargeOffMonthKey                            NaN                NaN   
ChargeOffMOB                                 NaN                NaN   
BalanceAtDefault                             NaN                NaN   
JointIndicator                               NaN                NaN   
...                                          ...                ...   
NumPriorCrescentApplicationsLast60Days       NaN                NaN   
NumPriorCrescentApplicationsLast90Days       NaN                NaN   
NumPriorCrescentApplicationsLast180Days      NaN                NaN   
JDPUVIDiffCOFromOrig                         NaN                NaN   
MoodysUVIDiffCOFromOrig                      NaN                NaN   

                                         ChargeOffMOB  BalanceAtDefault  \
Vintage                                     -0.373906          0.069838 

/var/folders/j2/9frmypq12fj32dw3sv8_k2r80000gn/T/ipykernel_7916/3076459492.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = x_corr.where(np.triu(np.ones(x_corr.shape),k=1).astype(np.bool))


In [11]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.75)]
print(); print(to_drop)


['FICOScorePctAvgFICOLast30DaysBookedLoans', 'VehicleManufacturerRebate', 'VehicleValueBlackBook', 'MoodysUsedVehicleIndex', 'MoodysNumEmployed', 'MoodysGDPReal', 'MoodysNumLaborForce', 'FinancedAmt', 'LoanAmtLine3', 'LoanAmtLine4', 'LTVLine4', 'LTVLine5', 'LTVLine3ExclRebate', 'LTVLine4ExclRebate', 'LTVLine5ExclRebate', 'CashDownPmtPctBBVehicleValue', 'CashDownPmtPctLine3', 'CashDownPmtPctLine4', 'CashDownPmtPctLine5', 'PaymentAmt', 'BackendTotalAmt', 'BackendTotalPctBBVehicleValue', 'BackendTotalPctBBVehicleValueGTE13PctIndicator', 'BackendTotalPctLine3', 'BackendTotalPctLine4', 'BackendTotalPctLine5', 'TotalDownPmtPctBBVehicleValue', 'TotalDownPmtPctLine3', 'TotalDownPmtPctLine4', 'TotalDownPmtPctLine5', 'ContractRate', 'BuyRate', 'NumExceptionPctAvgNumExceptionLast30DaysBookedLoans', 'NumPriorCrescentApplicationsLast60Days', 'NumPriorCrescentApplicationsLast90Days', 'NumPriorCrescentApplicationsLast180Days']


In [12]:
X_new = X.drop(X[to_drop], axis=1)

In [13]:
X_new = X_new.drop(['NumJobsApplicant'], axis = 1)

In [14]:
# Final Explanatory numeric variables 

X_new.head()

,Vintage,ChargeOffMonthKey,ChargeOffMOB,BalanceAtDefault,JointIndicator,LCPIndicator,FICOScore,ApplicationWeekday,NewUsedIndicator,VehicleAge,...,CreditBureauNumCollections,CreditBureauPctTradesDelq,NumExceptionALL,NumExceptionLTV,NumExceptionPTI,NumExceptionDTI,NumPriorCrescentApplications,NumPriorCrescentApplicationsLast30Days,JDPUVIDiffCOFromOrig,MoodysUVIDiffCOFromOrig
0,1.698842,1.067574,-0.299149,-1.298721,-0.498678,2.09376,0.348685,-0.312812,-0.689011,0.087601,...,-0.699069,0.689948,0.671895,9.993380,-0.134007,-0.105499,-0.554403,-0.434474,2.181875,2.043525
1,2.934303,1.067574,-1.023056,1.439201,-0.498678,-0.47761,3.340728,-1.442510,-0.689011,0.054092,...,-0.873341,-3.202200,-0.811378,-0.100066,-0.134007,-0.105499,-0.554403,-0.434474,2.020707,1.798437
2,-0.993162,-0.158740,0.941835,-1.617047,-0.498678,-0.47761,0.323112,0.816886,-0.689011,-0.012924,...,-0.001982,0.078359,-0.811378,-0.100066,-0.134007,-0.105499,-0.554403,-0.434474,-1.766859,-0.764104
3,-0.941142,-1.224601,0.011097,-0.061589,-0.498678,-0.47761,-1.390280,1.381735,1.451355,-0.079940,...,-0.873341,-0.074538,0.671895,-0.100066,-0.134007,-0.105499,0.352177,-0.434474,-1.024394,-0.725979
4,1.737857,1.021731,-1.023056,-0.187354,-0.498678,-0.47761,-0.674235,-0.312812,1.451355,-0.079940,...,1.043649,0.253274,-0.811378,-0.100066,-0.134007,-0.105499,0.352177,-0.434474,0.945700,1.545178


### 1C: Running the initial OLS on numeric explanatory variables and removing insignificant numeric variables

In [15]:
# Running the first OLS on continuous variables 

import statsmodels.api as sm

result = sm.OLS(Y, X_new).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          True_Recovery   R-squared (uncentered):                   0.191
Model:                            OLS   Adj. R-squared (uncentered):              0.189
Method:                 Least Squares   F-statistic:                              84.93
Date:                Sat, 29 Oct 2022   Prob (F-statistic):                        0.00
Time:                        20:52:39   Log-Likelihood:                         -18012.
No. Observations:               13718   AIC:                                  3.610e+04
Df Residuals:                   13680   BIC:                                  3.639e+04
Df Model:                          38                                                  
Covariance Type:            nonrobust                                                  
======================================================================================================================
                                                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Vintage                                               -0.0301      0.023     -1.293      0.196      -0.076       0.016
ChargeOffMonthKey                                      0.0737      0.023      3.193      0.001       0.028       0.119
ChargeOffMOB                                          -0.3307      0.024    -13.803      0.000      -0.378      -0.284
BalanceAtDefault                                       0.0252      0.012      2.163      0.031       0.002       0.048
JointIndicator                                         0.0586      0.009      6.845      0.000       0.042       0.075
LCPIndicator                                          -0.0225      0.009     -2.628      0.009      -0.039      -0.006
FICOScore                                              0.0058      0.009      0.680      0.497      -0.011       0.023
ApplicationWeekday                                    -0.0063      0.008     -0.817      0.414      -0.021       0.009
NewUsedIndicator                                      -0.0465      0.013     -3.505      0.000      -0.072      -0.020
VehicleAge                                            -0.0153      0.008     -1.989      0.047      -0.030      -0.000
VehicleMileage                                        -0.1194      0.018     -6.694      0.000      -0.154      -0.084
TradeInIndicator                                       0.0748      0.009      8.406      0.000       0.057       0.092
JDPowerUsedVehicleIndex                                0.0789      0.026      3.010      0.003       0.028       0.130
MoodysNumUnemployed                                   -0.0143      0.009     -1.672      0.094      -0.031       0.002
MoodysGasoline                                         0.0173      0.009      2.025      0.043       0.001       0.034
MoodysUnemploymentRate                                -0.0782      0.009     -8.464      0.000      -0.096      -0.060
LTVLine3                                              -0.1924      0.009    -20.281      0.000      -0.211      -0.174
CashDownPmt                                            0.0279      0.012      2.380      0.017       0.005       0.051
PaymentToFinancedAmtRatio                             -0.0797      0.015     -5.431      0.000      -0.108      -0.051
PTILine5                                               0.0292      0.009      3.319      0.001       0.012       0.046
BackendGapAmt                                          0.0302      0.008      3.754      0.000       0.014       0.046
BackendWarrantyAmt                                     0.0229      0.008      2.772      0.006       0.007       0.039
TotalDown

In [16]:
X_new = X_new.drop(['NumPriorCrescentApplicationsLast30Days','NumPriorCrescentApplications','NumExceptionDTI',
                   'NumExceptionPTI', 'NumExceptionLTV','NumExceptionALL','CreditBureauNumCollections',
                   'CreditBureauPctTradesDelq','CreditBureauNumTradesCurrent','CreditBureauNumTrades','DealerFeeNet',
                   'DealerReserve','TotalDownPmt','MoodysNumUnemployed','ApplicationWeekday','FICOScore','Vintage'], axis=1)

In [17]:
# Second Regression Model with Continuous Variables

result2 = sm.OLS(Y, X_new).fit()
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          True_Recovery   R-squared (uncentered):                   0.188
Model:                            OLS   Adj. R-squared (uncentered):              0.187
Method:                 Least Squares   F-statistic:                              151.4
Date:                Sat, 29 Oct 2022   Prob (F-statistic):                        0.00
Time:                        20:52:46   Log-Likelihood:                         -18033.
No. Observations:               13718   AIC:                                  3.611e+04
Df Residuals:                   13697   BIC:                                  3.627e+04
Df Model:                          21                                                  
Covariance Type:            nonrobust                                                  
======================================================================================================================
                                                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
ChargeOffMonthKey                                      0.0602      0.021      2.872      0.004       0.019       0.101
ChargeOffMOB                                          -0.3178      0.022    -14.277      0.000      -0.361      -0.274
BalanceAtDefault                                       0.0310      0.011      2.940      0.003       0.010       0.052
JointIndicator                                         0.0586      0.008      6.970      0.000       0.042       0.075
LCPIndicator                                          -0.0274      0.008     -3.356      0.001      -0.043      -0.011
NewUsedIndicator                                      -0.0458      0.013     -3.536      0.000      -0.071      -0.020
VehicleAge                                            -0.0153      0.008     -1.992      0.046      -0.030      -0.000
VehicleMileage                                        -0.1126      0.015     -7.410      0.000      -0.142      -0.083
TradeInIndicator                                       0.0806      0.008      9.731      0.000       0.064       0.097
JDPowerUsedVehicleIndex                                0.0926      0.024      3.866      0.000       0.046       0.140
MoodysGasoline                                         0.0189      0.008      2.246      0.025       0.002       0.035
MoodysUnemploymentRate                                -0.0720      0.009     -8.386      0.000      -0.089      -0.055
LTVLine3                                              -0.1954      0.009    -21.656      0.000      -0.213      -0.178
CashDownPmt                                            0.0390      0.009      4.309      0.000       0.021       0.057
PaymentToFinancedAmtRatio                             -0.0914      0.012     -7.872      0.000      -0.114      -0.069
PTILine5                                               0.0289      0.008      3.412      0.001       0.012       0.045
BackendGapAmt                                          0.0314      0.008      3.934      0.000       0.016       0.047
BackendWarrantyAmt                                     0.0257      0.008      3.149      0.002       0.010       0.042
EmploymentMonthsSinceHiredCurrentEmployerApplicant     0.0331      0.008      4.250      0.000       0.018       0.048
JDPUVIDiffCOFromOrig                                   0.0348      0.018      1.961      0.050    2.09e-05       0.070
MoodysUVIDiffCOFromOrig                                0.0807      0.016      4.968      0.000       0.049       0.113
==============================================================================
Omnibus:                      841.089   Durbin-Wa

In [18]:
X_new = sm.add_constant(X_new)

In [19]:
# 3rd OLS with constant
result3 = sm.OLS(Y, X_new).fit()
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          True_Recovery   R-squared:                       0.188
Model:                            OLS   Adj. R-squared:                  0.187
Method:                 Least Squares   F-statistic:                     151.4
Date:                Sat, 29 Oct 2022   Prob (F-statistic):               0.00
Time:                        20:52:50   Log-Likelihood:                -18033.
No. Observations:               13718   AIC:                         3.611e+04
Df Residuals:                   13696   BIC:                         3.628e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
======================================================================================================================
                                                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
const                                               2.514e-15      0.008   3.27e-13      1.000      -0.015       0.015
ChargeOffMonthKey                                      0.0602      0.021      2.872      0.004       0.019       0.101
ChargeOffMOB                                          -0.3178      0.022    -14.277      0.000      -0.361      -0.274
BalanceAtDefault                                       0.0310      0.011      2.940      0.003       0.010       0.052
JointIndicator                                         0.0586      0.008      6.970      0.000       0.042       0.075
LCPIndicator                                          -0.0274      0.008     -3.356      0.001      -0.043      -0.011
NewUsedIndicator                                      -0.0458      0.013     -3.536      0.000      -0.071      -0.020
VehicleAge                                            -0.0153      0.008     -1.992      0.046      -0.030      -0.000
VehicleMileage                                        -0.1126      0.015     -7.409      0.000      -0.142      -0.083
TradeInIndicator                                       0.0806      0.008      9.731      0.000       0.064       0.097
JDPowerUsedVehicleIndex                                0.0926      0.024      3.866      0.000       0.046       0.140
MoodysGasoline                                         0.0189      0.008      2.246      0.025       0.002       0.035
MoodysUnemploymentRate                                -0.0720      0.009     -8.386      0.000      -0.089      -0.055
LTVLine3                                              -0.1954      0.009    -21.656      0.000      -0.213      -0.178
CashDownPmt                                            0.0390      0.009      4.309      0.000       0.021       0.057
PaymentToFinancedAmtRatio                             -0.0914      0.012     -7.871      0.000      -0.114      -0.069
PTILine5                                               0.0289      0.008      3.412      0.001       0.012       0.045
BackendGapAmt                                          0.0314      0.008      3.933      0.000       0.016       0.047
BackendWarrantyAmt                                     0.0257      0.008      3.149      0.002       0.010       0.042
EmploymentMonthsSinceHiredCurrentEmployerApplicant     0.0331      0.008      4.250      0.000       0.018       0.048
JDPUVIDiffCOFromOrig                                   0.0348      0.018      1.961      0.050    1.96e-05       0.070
MoodysUVIDiffCOFromOrig                                0.0807      0.016      4.968      0.000       0.049       0.113
==============================================================================
Omnibus:                     

## Part 2: Dealing with categorical variables

### 2A: Understanding the frequency distribustions of categorical variables

In [ ]:
# Focusing on the Categorical Variables
cat_var.info()

In [ ]:
cat_var['StateApplicant'].value_counts()

In [ ]:
cat_var['EmploymentJobTypeApplicant'].value_counts()

In [ ]:
cat_var['BackendType'].value_counts()

In [ ]:
cat_var['VehicleModelNADA'].value_counts()

In [ ]:
cat_var['VehicleMakeNADA'].value_counts()

### 2B: Feature Engineering

In [20]:
from scipy.stats import chi2_contingency

# Chi-square test between StateApplicant & EmploymentJobTypeApplicant
# Null Hypothesis (H0): Relationship between the categorical variables don't exist (Independent)
# Alternate Hypothesis (H1): Relationship between the categorical variables exist (Dependent)

chisqt1 = pd.crosstab(cat_var['StateApplicant'], cat_var['EmploymentJobTypeApplicant'], margins=True)
c, p, dof, expected = chi2_contingency(chisqt1)
p

# StateApplicant & EmploymentJobTypeApplicant are Independent


0.05217724590144371

In [21]:
# Chi-square test between BackendType & VehicleMakeNADA
# Null Hypothesis (H0): Relationship between the categorical variables don't exist (Independent)
# Alternate Hypothesis (H1): Relationship between the categorical variables exist (Dependent)

chisqt2 = pd.crosstab(cat_var['BackendType'], cat_var['VehicleMakeNADA'], margins=True)
c, p, dof, expected = chi2_contingency(chisqt2)
p

# StateApplicant & EmploymentJobTypeApplicant are Dependent

0.02636247747190268

In [22]:
# Chi-square test between BackendType & VehicleModelNADA
# Null Hypothesis (H0): Relationship between the categorical variables don't exist (Independent)
# Alternate Hypothesis (H1): Relationship between the categorical variables exist (Dependent)

chisqt3 = pd.crosstab(cat_var['BackendType'], cat_var['VehicleModelNADA'], margins=True)
c, p, dof, expected = chi2_contingency(chisqt3)
p

# StateApplicant & EmploymentJobTypeApplicant are Independent

0.8891431128118706

In [23]:
# Chi-square test between VehicleModelNADA & VehicleMakeNADA
# Null Hypothesis (H0): Relationship between the categorical variables don't exist (Independent)
# Alternate Hypothesis (H1): Relationship between the categorical variables exist (Dependent)

chisqt4 = pd.crosstab(cat_var['VehicleModelNADA'], cat_var['VehicleMakeNADA'], margins=True)
c, p, dof, expected = chi2_contingency(chisqt4)
p

# StateApplicant & EmploymentJobTypeApplicant are Dependent

0.0

#### Final Categorical Variables are StateApplicant, EmploymentJobTypeApplicant, BackendType, VehicleModelNADA

### 2C: Data Imputation

In [ ]:
cat_var.info()

In [24]:
cat_var['StateApplicant'] = cat_var['StateApplicant'].fillna('TX')

/var/folders/j2/9frmypq12fj32dw3sv8_k2r80000gn/T/ipykernel_7916/1020850320.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_var['StateApplicant'] = cat_var['StateApplicant'].fillna('TX')


In [ ]:
cat_var['VehicleModelNADA'].mode()

In [25]:
cat_var['VehicleModelNADA'] = cat_var['VehicleModelNADA'].fillna('FORTE')

/var/folders/j2/9frmypq12fj32dw3sv8_k2r80000gn/T/ipykernel_7916/3364838033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_var['VehicleModelNADA'] = cat_var['VehicleModelNADA'].fillna('FORTE')


In [ ]:
cat_var['VehicleMakeNADA'].mode()

In [26]:
cat_var['VehicleMakeNADA'] = cat_var['VehicleMakeNADA'].fillna('NISSAN')

/var/folders/j2/9frmypq12fj32dw3sv8_k2r80000gn/T/ipykernel_7916/2297225397.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_var['VehicleMakeNADA'] = cat_var['VehicleMakeNADA'].fillna('NISSAN')


In [27]:
cat_var.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   StateApplicant              13718 non-null  object
 1   EmploymentJobTypeApplicant  13718 non-null  object
 2   BackendType                 13718 non-null  object
 3   VehicleModelNADA            13718 non-null  object
 4   VehicleMakeNADA             13718 non-null  object
 5   Loss_Date_150               13718 non-null  object
dtypes: object(6)
memory usage: 643.2+ KB


### 2D: One-hot encoding

In [28]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)
columns_to_one_hot = ['StateApplicant','BackendType','EmploymentJobTypeApplicant','VehicleMakeNADA','VehicleModelNADA']
encoded_array = enc.fit_transform(cat_var.loc[:,columns_to_one_hot])
df_encoded = pd.DataFrame(encoded_array,columns=enc.get_feature_names_out() )
df_sklearn_encoded = pd.concat([cat_var,df_encoded],axis=1)
df_sklearn_encoded

,StateApplicant,EmploymentJobTypeApplicant,BackendType,VehicleModelNADA,VehicleMakeNADA,Loss_Date_150,StateApplicant_AL,StateApplicant_AR,StateApplicant_AZ,StateApplicant_CA,...,VehicleModelNADA_WRX,VehicleModelNADA_X3 SERIES,VehicleModelNADA_X5 SERIES,VehicleModelNADA_XB,VehicleModelNADA_XF,VehicleModelNADA_XTERRA,VehicleModelNADA_XTS,VehicleModelNADA_XV CROSSTREK,VehicleModelNADA_YARIS,VehicleModelNADA_YUKON
0,MO,EMPLOYED,BackendWarranty,FOCUS,FORD,2018-11-05 00:00:00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GA,EMPLOYED,BackendGAP,CLA CLASS,MERCEDES-BENZ,2018-11-26 00:00:00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,FL,EMPLOYED,BackendNone,ALTIMA,NISSAN,2017-04-26 00:00:00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NM,EMPLOYED,BackendGAP,SOUL,KIA,2016-11-13 00:00:00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NC,OTHER,BackendGAP,RIO,KIA,2018-07-26 00:00:00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13713,FL,RETIRED,BackendNone,FORTE,NISSAN,2018-08-26 00:00:00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13714,WI,EMPLOYED,BackendNone,FORTE,NISSAN,2017-09-14 00:00:00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13715,AL,EMPLOYED,BackendWarranty,COROLLA,TOYOTA,2018-03-26 00:00:00,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13716,TX,EMPLOYED,BackendGAP,SORENTO,KIA,2016-12-06 00:00:00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
MC_cat = df_sklearn_encoded.drop(['StateApplicant', 'EmploymentJobTypeApplicant', 'BackendType', 'VehicleModelNADA', 'VehicleMakeNADA', 
                                  'Loss_Date_150'], axis = 1)

In [30]:
MC_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Columns: 307 entries, StateApplicant_AL to VehicleModelNADA_YUKON
dtypes: float64(307)
memory usage: 32.1 MB


### 2E: Multicollinearity check 

In [31]:
#Check the correlation between explanatory variables

x_corr = MC_cat.corr()

x_corr

,StateApplicant_AL,StateApplicant_AR,StateApplicant_AZ,StateApplicant_CA,StateApplicant_CO,StateApplicant_CT,StateApplicant_DC,StateApplicant_DE,StateApplicant_FL,StateApplicant_GA,...,VehicleModelNADA_WRX,VehicleModelNADA_X3 SERIES,VehicleModelNADA_X5 SERIES,VehicleModelNADA_XB,VehicleModelNADA_XF,VehicleModelNADA_XTERRA,VehicleModelNADA_XTS,VehicleModelNADA_XV CROSSTREK,VehicleModelNADA_YARIS,VehicleModelNADA_YUKON
StateApplicant_AL,1.000000,-0.011796,-0.027900,-0.003927,-0.028880,-0.003927,-0.013339,-0.005892,-0.070874,-0.040414,...,-0.003927,-0.003401,-0.003927,-0.003401,-0.004391,-0.002777,-0.002777,-0.002777,0.021476,-0.002777
StateApplicant_AR,-0.011796,1.000000,-0.006223,-0.000876,-0.006442,-0.000876,-0.002975,-0.001314,-0.015809,-0.009015,...,-0.000876,-0.000759,-0.000876,-0.000759,-0.000979,-0.000619,-0.000619,-0.000619,-0.002192,-0.000619
StateApplicant_AZ,-0.027900,-0.006223,1.000000,-0.002072,-0.015237,-0.002072,-0.007037,-0.003109,-0.037393,-0.021322,...,-0.002072,-0.001794,0.033637,-0.001794,-0.002317,-0.001465,-0.001465,-0.001465,-0.005184,-0.001465
StateApplicant_CA,-0.003927,-0.000876,-0.002072,1.000000,-0.002145,-0.000292,-0.000991,-0.000438,-0.005264,-0.003001,...,-0.000292,-0.000253,-0.000292,-0.000253,-0.000326,-0.000206,-0.000206,-0.000206,-0.000730,-0.000206
StateApplicant_CO,-0.028880,-0.006442,-0.015237,-0.002145,1.000000,-0.002145,-0.007285,-0.003218,-0.038706,-0.022071,...,0.032389,-0.001857,-0.002145,-0.001857,-0.002398,-0.001517,-0.001517,0.047318,-0.005366,-0.001517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VehicleModelNADA_XTERRA,-0.002777,-0.000619,-0.001465,-0.000206,-0.001517,-0.000206,-0.000700,-0.000309,-0.003722,-0.002122,...,-0.000206,-0.000179,-0.000206,-0.000179,-0.000231,1.000000,-0.000146,-0.000146,-0.000516,-0.000146
VehicleModelNADA_XTS,-0.002777,-0.000619,-0.001465,-0.000206,-0.001517,-0.000206,-0.000700,-0.000309,0.017729,-0.002122,...,-0.000206,-0.000179,-0.000206,-0.000179,-0.000231,-0.000146,1.000000,-0.000146,-0.000516,-0.000146
VehicleModelNADA_XV CROSSTREK,-0.002777,-0.000619,-0.001465,-0.000206,0.047318,-0.000206,-0.000700,-0.000309,-0.003722,-0.002122,...,-0.000206,-0.000179,-0.000206,-0.000179,-0.000231,-0.000146,-0.000146,1.000000,-0.000516,-0.000146
VehicleModelNADA_YARIS,0.021476,-0.002192,-0.005184,-0.000730,-0.005366,-0.000730,-0.002478,-0.001095,-0.001024,0.002516,...,-0.000730,-0.000632,-0.000730,-0.000632,-0.000816,-0.000516,-0.000516,-0.000516,1.000000,-0.000516


In [32]:
upper_tri = x_corr.where(np.triu(np.ones(x_corr.shape),k=1).astype(np.bool))
print(upper_tri)

                               StateApplicant_AL  StateApplicant_AR  \
StateApplicant_AL                            NaN          -0.011796   
StateApplicant_AR                            NaN                NaN   
StateApplicant_AZ                            NaN                NaN   
StateApplicant_CA                            NaN                NaN   
StateApplicant_CO                            NaN                NaN   
...                                          ...                ...   
VehicleModelNADA_XTERRA                      NaN                NaN   
VehicleModelNADA_XTS                         NaN                NaN   
VehicleModelNADA_XV CROSSTREK                NaN                NaN   
VehicleModelNADA_YARIS                       NaN                NaN   
VehicleModelNADA_YUKON                       NaN                NaN   

                               StateApplicant_AZ  StateApplicant_CA  \
StateApplicant_AL                      -0.027900          -0.003927   
State

/var/folders/j2/9frmypq12fj32dw3sv8_k2r80000gn/T/ipykernel_7916/3076459492.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = x_corr.where(np.triu(np.ones(x_corr.shape),k=1).astype(np.bool))


In [33]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.75)]
print(); print(to_drop)


['VehicleModelNADA_200', 'VehicleModelNADA_3 SERIES', 'VehicleModelNADA_500', 'VehicleModelNADA_A4', 'VehicleModelNADA_C CLASS', 'VehicleModelNADA_COOPER', 'VehicleModelNADA_EQUINOX', 'VehicleModelNADA_G37', 'VehicleModelNADA_GALANT', 'VehicleModelNADA_H3', 'VehicleModelNADA_MKZ', 'VehicleModelNADA_PASSAT', 'VehicleModelNADA_S60', 'VehicleModelNADA_XF']


In [34]:
MC_cat_new = MC_cat.drop(MC_cat[to_drop], axis=1)

In [35]:
MC_cat_new.head()

,StateApplicant_AL,StateApplicant_AR,StateApplicant_AZ,StateApplicant_CA,StateApplicant_CO,StateApplicant_CT,StateApplicant_DC,StateApplicant_DE,StateApplicant_FL,StateApplicant_GA,...,VehicleModelNADA_WRANGLER,VehicleModelNADA_WRX,VehicleModelNADA_X3 SERIES,VehicleModelNADA_X5 SERIES,VehicleModelNADA_XB,VehicleModelNADA_XTERRA,VehicleModelNADA_XTS,VehicleModelNADA_XV CROSSTREK,VehicleModelNADA_YARIS,VehicleModelNADA_YUKON
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
MC_cat_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Columns: 293 entries, StateApplicant_AL to VehicleModelNADA_YUKON
dtypes: float64(293)
memory usage: 30.7 MB


In [37]:
MC_cat_new.describe()

,StateApplicant_AL,StateApplicant_AR,StateApplicant_AZ,StateApplicant_CA,StateApplicant_CO,StateApplicant_CT,StateApplicant_DC,StateApplicant_DE,StateApplicant_FL,StateApplicant_GA,...,VehicleModelNADA_WRANGLER,VehicleModelNADA_WRX,VehicleModelNADA_X3 SERIES,VehicleModelNADA_X5 SERIES,VehicleModelNADA_XB,VehicleModelNADA_XTERRA,VehicleModelNADA_XTS,VehicleModelNADA_XV CROSSTREK,VehicleModelNADA_YARIS,VehicleModelNADA_YUKON
count,13718.000000,13718.000000,13718.000000,13718.000000,13718.000000,13718.000000,13718.000000,13718.000000,13718.000000,13718.000000,...,13718.000000,13718.000000,13718.000000,13718.000000,13718.000000,13718.000000,13718.000000,13718.000000,13718.000000,13718.000000
mean,0.050226,0.002624,0.014506,0.000292,0.015527,0.000292,0.003353,0.000656,0.086747,0.029961,...,0.000802,0.000292,0.000219,0.000292,0.000219,0.000146,0.000146,0.000146,0.001822,0.000146
std,0.218419,0.051162,0.119570,0.017074,0.123641,0.017074,0.057812,0.025606,0.281475,0.170485,...,0.028307,0.017074,0.014787,0.017074,0.014787,0.012074,0.012074,0.012074,0.042652,0.012074
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Part 3: Combining the numeric, categorical variables & dependent variable for further analysis

### 3A: Combining the data

In [38]:
X_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Data columns (total 22 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   const                                               13718 non-null  float64
 1   ChargeOffMonthKey                                   13718 non-null  float64
 2   ChargeOffMOB                                        13718 non-null  float64
 3   BalanceAtDefault                                    13718 non-null  float64
 4   JointIndicator                                      13718 non-null  float64
 5   LCPIndicator                                        13718 non-null  float64
 6   NewUsedIndicator                                    13718 non-null  float64
 7   VehicleAge                                          13718 non-null  float64
 8   VehicleMileage                                      13718 non-null  float64


In [39]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   True_Recovery  13718 non-null  float64
dtypes: float64(1)
memory usage: 107.3 KB


In [66]:
X_var = pd.concat([X_new, MC_cat_new], axis=1)

In [67]:
X_var = X_var.drop(['const'], axis = 1)

### 3B: Final Multicollinearity check

#### No Multicollinearity present as of now

### 3C: OLS Model with final cleaned data

In [68]:
# 4th OLS with constant
result4 = sm.OLS(Y,X_var).fit()
result4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          True_Recovery   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     13.84
Date:                Sat, 29 Oct 2022   Prob (F-statistic):               0.00
Time:                        21:06:45   Log-Likelihood:                -17683.
No. Observations:               13718   AIC:                         3.594e+04
Df Residuals:                   13429   BIC:                         3.812e+04
Df Model:                         288                                         
Covariance Type:            nonrobust                                         
======================================================================================================================
                                                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
ChargeOffMonthKey                                      0.0771      0.022      3.458      0.001       0.033       0.121
ChargeOffMOB                                          -0.3521      0.024    -14.877      0.000      -0.398      -0.306
BalanceAtDefault                                      -0.0351      0.013     -2.655      0.008      -0.061      -0.009
JointIndicator                                         0.0551      0.009      6.403      0.000       0.038       0.072
LCPIndicator                                          -0.0235      0.008     -2.859      0.004      -0.040      -0.007
NewUsedIndicator                                      -0.0336      0.014     -2.484      0.013      -0.060      -0.007
VehicleAge                                            -0.0152      0.008     -1.995      0.046      -0.030      -0.000
VehicleMileage                                        -0.1644      0.016    -10.068      0.000      -0.196      -0.132
TradeInIndicator                                       0.0702      0.008      8.419      0.000       0.054       0.087
JDPowerUsedVehicleIndex                                0.0982      0.024      4.096      0.000       0.051       0.145
MoodysGasoline                                         0.0153      0.008      1.820      0.069      -0.001       0.032
MoodysUnemploymentRate                                -0.0021      0.021     -0.102      0.919      -0.043       0.039
LTVLine3                                              -0.1688      0.009    -17.772      0.000      -0.187      -0.150
CashDownPmt                                            0.0380      0.009      4.059      0.000       0.020       0.056
PaymentToFinancedAmtRatio                             -0.0813      0.012     -6.699      0.000      -0.105      -0.058
PTILine5                                               0.0280      0.008      3.299      0.001       0.011       0.045
BackendGapAmt                                         -0.0048      0.027     -0.180      0.857      -0.057       0.047
BackendWarrantyAmt                                    -0.0068      0.032     -0.210      0.833      -0.070       0.057
EmploymentMonthsSinceHiredCurrentEmployerApplicant     0.0242      0.008      3.055      0.002       0.009       0.040
JDPUVIDiffCOFromOrig                                   0.0449      0.018      2.500      0.012       0.010       0.080
MoodysUVIDiffCOFromOrig                                0.0690      0.016      4.222      0.000       0.037       0.101
StateApplicant_AL                                     -0.1107      0.061     -1.811      0.070      -0.231       0.009
StateApplicant_AR                                     -0.1703      0.153     -1.111      0.266      -0.471  

### 3D: Removing Insignificant variables

In [69]:
import statsmodels.api as sm
import pandas as pd
import operator

def remove_most_insignificant(df, results):
    # use operator to find the key which belongs to the maximum value in the dictionary:
    max_p_value = max(results.pvalues.iteritems(), key=operator.itemgetter(1))[0]
    # this is the feature you want to drop:
    df.drop(columns = max_p_value, inplace = True)
    return df

insignificant_feature = True
while insignificant_feature:
        model = sm.OLS(Y, X_var)
        results = model.fit()
        significant = [p_value < 0.05 for p_value in results.pvalues]
        if all(significant):
            insignificant_feature = False
        else:
            if X_var.shape[1] == 1:  # if there's only one insignificant variable left
                print('No significant features found')
                results = None
                insignificant_feature = False
            else:            
                X_var = remove_most_insignificant(X_var, results)
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:          True_Recovery   R-squared (uncentered):                   0.218
Model:                            OLS   Adj. R-squared (uncentered):              0.214
Method:                 Least Squares   F-statistic:                              53.64
Date:                Sat, 29 Oct 2022   Prob (F-statistic):                        0.00
Time:                        21:09:14   Log-Likelihood:                         -17777.
No. Observations:               13718   AIC:                                  3.570e+04
Df Residuals:                   13647   BIC:                                  3.623e+04
Df Model:                          71                                                  
Covariance Type:            nonrobust                                                  
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-

## Part 4: Splitting the data into test and train to calculate the final MSE

### 4A: Combining the final data

In [70]:
insignificant_feature = True
while insignificant_feature:
        model = sm.OLS(Y, X_var)
        results = model.fit()
        significant = [p_value < 0.05 for p_value in results.pvalues]
        if all(significant):
            insignificant_feature = False
        else:
            if X_var.shape[1] == 1:  # if there's only one insignificant variable left
                print('No significant features found')
                results = None
                insignificant_feature = False
            else:            
                X_var = remove_most_insignificant(X_var, results)
print(X_var)

       ChargeOffMonthKey  ChargeOffMOB  JointIndicator  LCPIndicator  \
0               1.067574     -0.299149       -0.498678       2.09376   
1               1.067574     -1.023056       -0.498678      -0.47761   
2              -0.158740      0.941835       -0.498678      -0.47761   
3              -1.224601      0.011097       -0.498678      -0.47761   
4               1.021731     -1.023056       -0.498678      -0.47761   
...                  ...           ...             ...           ...   
13713           1.033192      1.045250        2.005300       2.09376   
13714          -0.101435     -1.023056       -0.498678      -0.47761   
13715           0.975887      0.217928        2.005300      -0.47761   
13716          -1.213141      0.114512       -0.498678      -0.47761   
13717          -2.405072     -1.126472       -0.498678      -0.47761   

       NewUsedIndicator  VehicleMileage  TradeInIndicator  \
0             -0.689011       -0.245309         -0.647344   
1            

In [71]:
X_var.head()

,ChargeOffMonthKey,ChargeOffMOB,JointIndicator,LCPIndicator,NewUsedIndicator,VehicleMileage,TradeInIndicator,JDPowerUsedVehicleIndex,MoodysGasoline,LTVLine3,...,VehicleModelNADA_RAM 2500,VehicleModelNADA_SEDONA,VehicleModelNADA_SORENTO,VehicleModelNADA_SOUL,VehicleModelNADA_TITAN,VehicleModelNADA_TOWN & COUNTRY,VehicleModelNADA_TUCSON,VehicleModelNADA_TUNDRA,VehicleModelNADA_VELOSTER,VehicleModelNADA_VERSA
0,1.067574,-0.299149,-0.498678,2.09376,-0.689011,-0.245309,-0.647344,-1.834283,0.010916,0.780873,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.067574,-1.023056,-0.498678,-0.47761,-0.689011,0.826260,-0.647344,-1.633009,1.823101,-0.501819,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.158740,0.941835,-0.498678,-0.47761,-0.689011,1.027366,1.544773,1.023568,1.904322,-1.460688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.224601,0.011097,-0.498678,-0.47761,1.451355,-1.134640,1.544773,1.037204,0.619549,-0.701780,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.021731,-1.023056,-0.498678,-0.47761,1.451355,-1.105057,-0.647344,-0.911506,0.944961,0.022489,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
MSE_data = pd.concat([X_var, Y], axis=1)

In [73]:
MSE_data.head()

,ChargeOffMonthKey,ChargeOffMOB,JointIndicator,LCPIndicator,NewUsedIndicator,VehicleMileage,TradeInIndicator,JDPowerUsedVehicleIndex,MoodysGasoline,LTVLine3,...,VehicleModelNADA_SEDONA,VehicleModelNADA_SORENTO,VehicleModelNADA_SOUL,VehicleModelNADA_TITAN,VehicleModelNADA_TOWN & COUNTRY,VehicleModelNADA_TUCSON,VehicleModelNADA_TUNDRA,VehicleModelNADA_VELOSTER,VehicleModelNADA_VERSA,True_Recovery
0,1.067574,-0.299149,-0.498678,2.09376,-0.689011,-0.245309,-0.647344,-1.834283,0.010916,0.780873,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.110074
1,1.067574,-1.023056,-0.498678,-0.47761,-0.689011,0.826260,-0.647344,-1.633009,1.823101,-0.501819,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.437607
2,-0.158740,0.941835,-0.498678,-0.47761,-0.689011,1.027366,1.544773,1.023568,1.904322,-1.460688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.911105
3,-1.224601,0.011097,-0.498678,-0.47761,1.451355,-1.134640,1.544773,1.037204,0.619549,-0.701780,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.363424
4,1.021731,-1.023056,-0.498678,-0.47761,1.451355,-1.105057,-0.647344,-0.911506,0.944961,0.022489,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.630527


### 4B: Splitting the data into test and training set

In [74]:
#Model development package
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#Model Evaluation package
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

In [75]:
#Dependent Variable
y = MSE_data[['True_Recovery']]

# Independent Variables
X = MSE_data.iloc[:,:-1]

In [76]:
# Splitting the Data into Train and test

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size= 0.7, train_size= 0.3,random_state= 42)


### 4C: Running the final Linear Regression model to calculate the MSE

In [77]:
#Running Simple Linear Regression Model and getting the coefficient with intercept

linreg = LinearRegression()

linreg.fit(X_train,y_train)

LinearRegression()

In [79]:
#testing the developed model in testing set
y_test_pred = linreg.predict(X_test)


In [83]:
#Evaluating the model based on MSE

#MSE
print(mse(y_test,y_test_pred))

1.4969598238316776e+23
